In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

In [2]:
posts_df = pd.read_csv('data/posts.csv')
posts_df.head()

,Unnamed: 0,age,comment,likes/views,link,promo,user
0,0,2 DAYS AGO,found my new favorite park!,"405,059 likes",/BytNlrQhRx8/,0,chrissyteigen
1,1,3 DAYS AGO,Happy bebe!,"1,739,218",/Byqz8uZh73s/,0,chrissyteigen
2,2,5 DAYS AGO,coated in a paste of fresh garlic and filled w...,"2,931,603",/BymErW1B9eL/,0,chrissyteigen
3,3,5 DAYS AGO,this kid,"371,095 likes",/Byl-aHjBXFX/,0,chrissyteigen
4,4,JUNE 8,home tomorrow 😩,"859,039 likes",/ByduG0BB_A3/,0,chrissyteigen


In [3]:
posts_df.shape

(1065, 7)

In [4]:
target = posts_df.promo
data = posts_df['comment'].map(word_tokenize).values

In [5]:
total_vocabulary = set(word for comment in data for word in comment)
print("There are {} unique tokens in our dataset.".format(len(total_vocabulary)))

There are 8994 unique tokens in our dataset.


In [6]:
glove = {}
with open('data/glove_data/glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

FileNotFoundError: [Errno 2] No such file or directory: 'data/glove_data/glove.6B.50d.txt'

In [7]:
glove['cool']

array([-0.65575 ,  0.45659 , -0.16748 , -0.58345 , -0.23073 , -0.78348 ,
       -0.23166 , -0.022452, -0.57968 ,  0.526   , -0.2214  ,  0.17614 ,
        0.46513 ,  0.79142 ,  0.017403,  1.0879  ,  0.24418 ,  0.27523 ,
       -0.26452 , -1.0389  ,  0.014045,  0.68459 ,  0.98151 ,  0.21561 ,
        0.36278 , -0.51819 , -0.40552 ,  1.349   ,  1.5399  ,  0.60541 ,
        2.6604  ,  0.074535, -0.076292,  0.12501 , -0.026268,  0.16843 ,
       -0.41844 ,  0.44505 ,  0.25033 , -1.1557  ,  0.24575 ,  0.41847 ,
       -0.10633 , -0.28433 ,  0.51215 ,  0.51371 ,  0.53004 , -0.889   ,
        0.054744,  0.78793 ], dtype=float32)

#### Create Mean Word Embeddings

In [8]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

#### Using Pipelines

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [10]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [11]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
/Users/sherzyang/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sherzyang/anaconda3/envs/learn-env/lib/python3.6/site-p

In [12]:
scores

[('Random Forest', 0.6281722128962179),
 ('Support Vector Machine', 0.5624321121753728),
 ('Logistic Regression', 0.5877851288634344)]

#### Deep Learning with word embeddings

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.preprocessing import text, sequence

In [14]:
y = pd.get_dummies(target).values

In [15]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(posts_df.comment))
list_tokenized_headlines = tokenizer.texts_to_sequences(posts_df.comment)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [16]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [17]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(20000, embedding_size)(input_)
x = Bidirectional(LSTM(25, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 2 different possible classes, so we use 2 neurons in our output layer
x = Dense(2, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 50)           30800     
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102   

In [20]:
model.fit(X_t, y, epochs=2, batch_size=32, validation_split=0.1)

W0620 10:01:48.142514 140735550342016 deprecation.py:323] From /Users/kaylischulz/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 958 samples, validate on 107 samples
Epoch 1/2
958/958 [==============================] - 5s 5ms/sample - loss: 0.6847 - accuracy: 0.5585 - val_loss: 0.6643 - val_accuracy: 0.6075
Epoch 2/2
958/958 [==============================] - 3s 3ms/sample - loss: 0.6453 - accuracy: 0.6253 - val_loss: 0.6220 - val_accuracy: 0.6449


In [21]:
model.fit(X_t, y, epochs=10, batch_size=32, validation_split=0.1)

Train on 958 samples, validate on 107 samples
Epoch 1/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.4526 - accuracy: 0.8299 - val_loss: 0.5988 - val_accuracy: 0.7009
Epoch 2/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.1945 - accuracy: 0.9436 - val_loss: 0.7486 - val_accuracy: 0.7103
Epoch 3/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.0814 - accuracy: 0.9802 - val_loss: 0.9855 - val_accuracy: 0.7196
Epoch 4/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.0373 - accuracy: 0.9927 - val_loss: 1.1748 - val_accuracy: 0.7290
Epoch 5/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.0219 - accuracy: 0.9948 - val_loss: 1.2374 - val_accuracy: 0.7196
Epoch 6/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.0079 - accuracy: 0.9979 - val_loss: 1.5036 - val_accuracy: 0.7196
Epoch 7/10
958/958 [==============================] - 3s 3ms/sample - loss: 0.0061 - accur

In [22]:
import pickle 
test_df = pickle.load(open('data/testing_posts.pkl', 'rb'))

In [24]:
list_tokenized_headlines = tokenizer.texts_to_sequences(test_df['comment'].map(word_tokenize).values)
testing_X = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [25]:
testing_y = pd.get_dummies(test_df['promo']).values

In [26]:
model.evaluate(testing_X, testing_y, verbose=True)

248/248 [==============================] - 1s 2ms/sample - loss: 1.9260 - accuracy: 0.7097


[1.925971269607544, 0.7096774]